## Automatic differentiation with ``torch.autograd``

When training neural networks, the most frequently used algorithm is
**back propagation**. In this algorithm, parameters (model weights) are
adjusted according to the **gradient** of the loss function with respect
to the given parameter.  The loss function calulates the difference between the expected output and the actual output that a neural network produces.  The goal is to get the result of the loss function as close to zero as possible.  The algorithm traverse backwards through the network network to adjust the weights and bias to retrain the model. That's why it's called back propagation. This back and forward process of retraining the model over time to reduce the loss to 0 is called the gradient descent.

To compute those gradients, PyTorch has a built-in differentiation engine
called `torch.autograd`. It supports automatic computation of gradient for any
computational graph.

Consider the simplest one-layer neural network, with input `x`,
parameters `w` and `b`, and some loss function. It can be defined in
PyTorch in the following manner:

在训练神经网络时，最常用的算法是
**反向传播**。 在这个算法中，参数（模型权重）是
根据损失函数的**梯度**调整
到给定的参数。 损失函数计算神经网络产生的预期输出和实际输出之间的差异。 目标是让损失函数的结果尽可能接近于零。 该算法通过网络网络向后遍历以调整权重和偏差以重新训练模型。 这就是为什么它被称为反向传播。 这种随着时间的推移重新训练模型以将损失减少到 0 的前后过程称为梯度下降。

为了计算这些梯度，PyTorch 有一个内置的微分引擎
称为 `torch.autograd`。 它支持自动计算任何梯度
计算图。

考虑最简单的一层神经网络，输入为``x``，
参数 `w` 和 `b`，以及一些损失函数。 它可以定义在
PyTorch 通过以下方式：

In [ ]:
%matplotlib inline
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

## Tensors, Functions and Computational graph

This code defines the following **computational graph**:

这段代码定义了如下**计算图**

<img alt="Diagram showing a computational graph with two parameters 'w' and 'b' to compute the gradients of loss." src="images/5-autograd-1.png" />

In this network, `w` and `b` are **parameters**, which we need to
optimize. Thus, we need to be able to compute the gradients of loss
function with respect to those variables. In order to do that, we set
the `requires_grad` property of those tensors.

> **Note:** You can set the value of `requires_grad` when creating a tensor, or later by using `x.requires_grad_(True)` method.

A function that we apply to tensors to construct computational graph is
in fact an object of class `Function`. This object knows how to
compute the function in the *forward* direction, and also how to compute
its derivative during the *backward propagation* step. A reference to
the backward propagation function is stored in `grad_fn` property of a
tensor. 

在这个网络中，`w` 和 `b` 是**参数**，我们需要
优化。 因此，我们需要能够计算损失的梯度
关于这些变量的函数。 为了做到这一点，我们设置
这些张量的 `requires_grad` 属性。

> **注意：**您可以在创建张量时设置 `requires_grad` 的值，或者稍后使用 `x.requires_grad_(True)` 方法。

我们应用于张量以构建计算图的函数是
实际上是``Function``类的对象。 这个对象知道如何
计算*forward*方向的函数，以及如何计算
它在*backward*步骤中的导数。 参考
反向传播函数存储在 a 的 `grad_fn` 属性中
张量。

In [ ]:
print('Gradient function for z =',z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x00000280CC630CA0>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward object at 0x00000280CC630310>


## Computing gradients

To optimize weights of parameters in the neural network, we need to
compute the derivatives of our loss function with respect to parameters,
namely, we need $\frac{\partial loss}{\partial w}$ and
$\frac{\partial loss}{\partial b}$ under some fixed values of
`x` and `y`. To compute those derivatives, we call
`loss.backward()`, and then retrieve the values from `w.grad` and
`b.grad`:

为了优化神经网络中的参数权重，我们需要
计算我们的损失函数相对于参数的导数。
即，我们需要$\frac{\partial loss}{\partial w}$ 和
$\frac{\partial loss}{\partial b}$ 在某些固定值下的
`x'和`y`的值。为了计算这些导数，我们调用
`loss.backward()`，然后从`w.grad`和`b.grad`中检索出数值。
`b.grad'。

In [ ]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.2739, 0.0490, 0.3279],
        [0.2739, 0.0490, 0.3279],
        [0.2739, 0.0490, 0.3279],
        [0.2739, 0.0490, 0.3279],
        [0.2739, 0.0490, 0.3279]])
tensor([0.2739, 0.0490, 0.3279])


> **Note:** We can only obtain the `grad` properties for the leaf nodes of the computational graph, which have `requires_grad` property set to ``True``. For all other nodes in our graph, gradients will not be available. In addition, we can only perform gradient calculations using ``backward`` once on a given graph, for performance reasons. If we need to do several ``backward`` calls on the same graph, we need to pass ``retain_graph=True`` to the ``backward`` call.

> **注意：** 我们只能获取计算图的叶节点的 `grad` 属性，其中 `requires_grad` 属性设置为 ``True``。 对于我们图表中的所有其他节点，梯度将不可用。 此外，出于性能原因，我们只能在给定的图形上使用 ``backward``执行一次梯度计算。 如果我们需要对同一个图进行多次``backward``调用，我们需要将``retain_graph=True``传递给``backward``调用。

## Disabling gradient tracking

By default, all tensors with `requires_grad=True` are tracking their
computational history and support gradient computation. However, there
are some cases when we do not need to do that, for example, when we have
trained the model and just want to apply it to some input data, i.e. we
only want to do *forward* computations through the network. We can stop
tracking computations by surrounding our computation code with
`torch.no_grad()` block:


默认情况下，所有 ``requires_grad=True``的张量都在跟踪其计算历史。
计算历史并支持梯度计算。然而，在某些情况下
在某些情况下，我们不需要这样做，例如，当我们已
训练好的模型，只是想把它应用于一些输入数据，也就是说，我们
即我们只想通过网络进行*前向计算。我们可以停止
我们可以通过在我们的计算代码周围添加以下内容来停止跟踪计算
`torch.no_grad()`块来停止跟踪计算。

In [ ]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


Another way to achieve the same result is to use the ``detach()`` method
on the tensor:




In [ ]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


There are reasons you might want to disable gradient tracking:
  - To mark some parameters in your neural network at **frozen parameters**. This is
    a very common scenario for fine tuning a pre-trained network.
  - To **speed up computations** when you are only doing forward pass, because computations on tensors that do
    not track gradients would be more efficient.

有一些原因你可能想禁用梯度跟踪。
  - 将你的神经网络中的一些参数标记为**冻结参数**。这是
    这是对预训练网络进行微调的一个非常常见的情况。
  - 当你只做正向传递时，为了**加速计算，因为对不跟踪梯度的张量的计算
    不跟踪梯度的张量的计算会更有效率。



More on Computational Graphs
----------------------------
Conceptually, autograd keeps a record of data (tensors) and all executed
operations (along with the resulting new tensors) in a directed acyclic
graph (DAG) consisting of
[Function](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)
objects. In this DAG, leaves are the input tensors, roots are the output
tensors. By tracing this graph from roots to leaves, you can
automatically compute the gradients using the chain rule.

In a forward pass, autograd does two things simultaneously:

- run the requested operation to compute a resulting tensor
- maintain the operation’s *gradient function* in the DAG.

The backward pass kicks off when `.backward()` is called on the DAG
root. `autograd` then:

- computes the gradients from each `.grad_fn`,
- accumulates them in the respective tensor’s `.grad` attribute
- using the chain rule, propagates all the way to the leaf tensors.

**DAGs are dynamic in PyTorch**

  An important thing to note is that the graph is recreated from scratch; after each
  `.backward()` call, autograd starts populating a new graph. This is
  exactly what allows you to use control flow statements in your model;
  you can change the shape, size and operations at every iteration if
  needed.



  
从概念上讲，autograd将数据（张量）的记录和所有执行的
和所有执行的操作（以及产生的新张量）记录在一个有向无环图（DAG）中。
图（DAG）中，包括
[函数](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)
对象。在这个DAG中，叶子是输入的张量，根部是输出的
张量。通过追踪这个图从根到叶，你可以
使用链式规则自动计算梯度。

在一个前向传递中，autograd同时做两件事。

- 运行所请求的操作，计算出一个结果张量
- 在DAG中维护该操作的*梯度函数*。

当在DAG根上调用`.backward()`时，后向传递开始。
根。`autograd`然后。

- 计算每个`.grad_fn'的梯度。
- 将它们累积到各自张量的`.grad'属性中。
- 使用连锁规则，一直传播到叶子张量。

**DAG在PyTorch中是动态的**。

  需要注意的是，图是从头开始重新创建的；在每次
  `.backward()`调用后，autograd开始填充一个新图。这就是
  这正是允许你在你的模型中使用控制流语句的原因。
  如果需要，你可以在每次迭代中改变形状、大小和操作。
  需要。



## Optional reading: Tensor gradients and Jacobian products

In many cases, we have a scalar loss function, and we need to compute
the gradient with respect to some parameters. However, there are cases
when the output function is an arbitrary tensor. In this case, PyTorch
allows you to compute so-called **Jacobian product**, and not the actual
gradient.

For a vector function $\vec{y}=f(\vec{x})$, where
$\vec{x}=\langle x_1,\dots,x_n\rangle$ and
$\vec{y}=\langle y_1,\dots,y_m\rangle$, a gradient of
$\vec{y}$ with respect to $\vec{x}$ is given by **Jacobian
matrix**, whose element $J_{ij}$ contains $\frac{\partial y_{i}}{\partial x_{j}}$.

Instead of computing the Jacobian matrix itself, PyTorch allows you to
compute **Jacobian Product** $v^T\cdot J$ for a given input vector
$v=(v_1 \dots v_m)$. This is achieved by calling `backward` with
$v$ as an argument. The size of $v$ should be the same as
the size of the original tensor, with respect to which we want to
compute the product:

在许多情况下，我们有一个标量损失函数，而我们需要计算
梯度与一些参数的关系。然而，也有一些情况
当输出函数是一个任意的张量时。在这种情况下，PyTorch
允许你计算所谓的**雅各布式乘积，而不是计算实际的
梯度。

对于一个矢量函数$vec{y}=f(\vec{x})$，其中
$\vec{x}=\langle x_1,\dots,x_n\rangle$ 和
$\vec{y}=\langle y_1,\dots,y_m\rangle$，一个梯度为
$\vec{y}$相对于$\vec{x}$的梯度由**乔布斯矩阵给出。
矩阵**，其元素$J_{ij}$包含$\frac{\partial y_{i}}{\partial x_{j}}$。

PyTorch允许你不计算雅各布矩阵本身，而是
对于一个给定的输入矢量$v=(v_1 \dots v_m)$，计算**雅各布系数** $v^T\cdot J$
。这是通过调用`backward`来实现的，参数为
$v$作为一个参数。v$的大小应该与
原始张量的大小，相对于它，我们想要
计算乘积。




In [ ]:
inp = torch.eye(5, requires_grad=True)
out = (inp+1).pow(2)
out.backward(torch.ones_like(inp), retain_graph=True)
print("First call\n", inp.grad)
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nSecond call\n", inp.grad)
inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nCall after zeroing gradients\n", inp.grad)

First call
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])


Notice that when we call `backward` for the second time with the same
argument, the value of the gradient is different. This happens because
when doing `backward` propagation, PyTorch **accumulates the
gradients**, i.e. the value of computed gradients is added to the
`grad` property of all leaf nodes of computational graph. If you want
to compute the proper gradients, you need to zero out the `grad`
property before. In real-life training an *optimizer* helps us to do
this.

请注意，当我们第二次调用 `backward` 时
参数，梯度的值是不同的。 发生这种情况是因为
在进行“backward”传播时，PyTorch **累积
梯度**，即计算梯度的值被添加到
计算图所有叶节点的“grad”属性。 如果你想
要计算适当的梯度，您需要将 `grad` 归零
之前的财产。 在现实生活中的训练中，*optimizer*帮助我们做到
这。


> **Note:** previously we were calling `backward()` function without parameters. This is equivalent to calling `backward(torch.tensor(1.0))`, which is a useful way to compute the gradients in case of a scalar-valued function, such as loss during neural network training.


> **注意：** 之前我们调用不带参数的 `backward()` 函数。 这相当于调用 `backward(torch.tensor(1.0))`，这是在标量值函数的情况下计算梯度的有用方法，例如神经网络训练期间的损失。


--------------


